# Data enrichment

### <u>Content:<u>

[1) Load data sets

[2) Add week day and month to reservation data
    
[3) Estimate part of the Swiss population that is on holiday for any given date
    
[4) Add coordinates for start and destination 

[5) Get weather data
    
[6) Get capacity data for trains in 2020 according to the expected train formation


In [93]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns
import scipy
from scipy.spatial.distance import cdist
import dateutil
rcParams['figure.figsize']=10,8

#### 1) Load data (#load_data) 

In [99]:
#read the reservation data 
df = pd.read_csv('data_raw/reservation_data_2019-2021_incl_capacity.csv', 
                 parse_dates=["date"], date_parser=lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S"))         
        
print('reservations: ')
display(df.tail(1))

# read the holiday data, include a date range column
df_schulferien = pd.read_csv('data_preprocessed/Schulferien.csv', dtype={"canton": "string", "population": "int32"})
df_schulferien['start'] = pd.to_datetime(df_schulferien['holidays_start'])
df_schulferien['end'] = pd.to_datetime(df_schulferien['holidays_end'])
df_schulferien = df_schulferien.drop(columns=["holidays_start", "holidays_end"])

print('schulferien: ')
display(df_schulferien.tail(1))

# train station coordinates data
df_coordinates = pd.read_csv('data_preprocessed/dienststellen.csv')
df_coordinates = df_coordinates[["abk_bahnhof", "lat", "lon"]]
df_coordinates = df_coordinates[df_coordinates['abk_bahnhof'].notna()]
display(df_coordinates.tail(1))

# weather data
df_weather = pd.read_csv("data_preprocessed/weather.csv")
#                        parse_dates=["validdate"], date_parser=lambda x: pd.to_datetime(x, format="%Y-%m-%d%T%H:%M:%S%Z"))         

#df_weather['date'] = df_weather['validdate'].apply(dateutil.parser.parse)
display(df_weather.tail(1))

# jahresformation
df_jahresformation = pd.read_csv("data_preprocessed/jahresformation.csv", dtype={"Block Bezeichnung": "string"})
display(df_jahresformation.tail(1))
df_jahresformation.dtypes

# kapazität
df_kapazität = pd.read_csv("data_preprocessed/rollmaterial-matching.csv")
display(df_kapazität.tail(1))


reservations: 


,Unnamed: 0,res_id,res_dt,date,train_nr,line,reserved,capacity,bp_from,bp_to,dep_ist,dep_soll,arr_ist,arr_soll,res_delta_ist,res_delta_soll,res_delta_valid
226735,226735,246187,2021-10-31 17:16:00,2021-10-31,1635,IC 51,1,6.0,BI,DMT,2021-10-31 19:49:00,2021-10-31 19:49:00,2021-10-31 20:17:00,2021-10-31 20:18:00,9216.0,9210,True


schulferien: 


,canton,population,start,end
161,national,7917100,2022-12-25,2022-12-26


C:\Users\tabea\miniconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,abk_bahnhof,lat,lon
47113,BRIA,47.260083,7.942674


,station_id,validdate,leisure_biking:idx,snow_depth:cm,t_2m:C,precip_24h:mm,weather_symbol_1h:idx,effective_cloud_cover:octas
4745,station_id,validdate,leisure_biking:idx,snow_depth:cm,t_2m:C,precip_24h:mm,weather_symbol_1h:idx,effective_cloud_cover:octas


C:\Users\tabea\miniconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Zug,ZNS,VP,Debicode,Zugart,Suf.,Traktion,Bhf von,Ländercode UIC,von,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
177418,96765,4,-162,1604,S,NaN,NaN,RUS,85,21:42:12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Block Bezeichnung in Jahresformation Fpl-2022,No. of hooks
36,Re460 AS4 A4(LBT) Bpm61 3B4(LBT) Bt4,10


#### 2) Week day <a name="stat"></a>

- Add a feature for weekday: 'weekday' and 'month'

In [4]:
df['weekday'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month

#### 3) Holidays <a name="stat"></a>
- Add a feature for number of people in holiday canton: n_holiday

In [8]:
# for each date, get the number of people in Switzerland who 
# are either on school holiday or national holiday..
def get_holiday_people(date):
    filtered_holidays = df_schulferien[(df_schulferien['start']<=date)&(df_schulferien['end']>=date)]
    is_national_holiday = (filtered_holidays["canton"]=="national").sum()
    if is_national_holiday:
        people = 7917100
    elif not(filtered_holidays.empty):
        filtered_holidays = filtered_holidays[filtered_holidays["canton"]!="national"]
        people = sum(filtered_holidays["population"])
    else: people=0
    return people

In [10]:
# filter df, only 2021 data
df=df[df['date']>='2021-01-01']
df['holiday_people'] = df.apply(lambda row : get_holiday_people(row['date']), axis = 1)


#### 4) Coordinates <a name="hr"></a>

In [15]:
# full join for start train station 
df = pd.merge(df, df_coordinates, left_on='bp_from', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_from", "lon": "lon_from"})

# full join for destination
df = pd.merge(df, df_coordinates, left_on='bp_to', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_to", "lon": "lon_to"})
display(df.head())

,Unnamed: 0,res_id,res_dt,date,train_nr,line,reserved,capacity,bp_from,bp_to,...,res_delta_ist,res_delta_soll,res_delta_valid,weekday,month,holiday_people,lat_from,lon_from,lat_to,lon_to
0,76737,83783,21/03/2021 00:00,2021-03-21,333,IC 6/61,1,NaN,BS,BN,...,26931.0,26922,False,6,3,0,47.547412,7.589563,46.948832,7.439131
1,76974,84080,21/03/2021 00:00,2021-03-21,979,IC 6/61,2,12.0,BS,BN,...,61117.0,61110,False,6,3,0,47.547412,7.589563,46.948832,7.439131
2,76975,84081,21/03/2021 00:00,2021-03-21,979,IC 6/61,1,12.0,BS,BN,...,61117.0,61110,False,6,3,0,47.547412,7.589563,46.948832,7.439131
3,76992,84098,21/03/2021 00:00,2021-03-21,1067,IC 6/61,1,12.0,BS,BN,...,37764.0,37710,False,6,3,0,47.547412,7.589563,46.948832,7.439131
4,77008,84115,15/03/2021 00:00,2021-03-21,1085,IC 6/61,1,12.0,BS,BN,...,588537.0,588510,False,6,3,0,47.547412,7.589563,46.948832,7.439131


#### 5) Weather <a name="corr"></a>

#### 6) Capacity <a name="corr"></a>

 - Für jede Reservation: Zugnummer im Jahresformation-Datensatz abrufen
 - Beachte: richtiges Jahr wählen, häufigste Formation
 - entsprechende Kapazität auslesen

In [96]:
list_kapazitäten=df_kapazität["Block Bezeichnung in Jahresformation Fpl-2022"].tolist()
df_jahresformation = df_jahresformation[df_jahresformation["Block Bezeichnung"].isin(list_kapazitäten)]
df_jahresformation=df_jahresformation[["Block Bezeichnung", "Zug", "Beginn Fahrplanperiode"]]

In [98]:
df_jahresformation = pd.merge(df_jahresformation, df_kapazität, left_on='Block Bezeichnung', right_on='Block Bezeichnung in Jahresformation Fpl-2022').drop(columns=["Block Bezeichnung in Jahresformation Fpl-2022"])
df_jahresformation.tail(2)

,Block Bezeichnung,Zug,Beginn Fahrplanperiode,No. of hooks_x,No. of hooks_y
31704,Re460 A4(hp)2Apm Bpm 2B4 2B4(78) Bt4 GBT,2682,Fpl-2022,16,16
31705,Re460 A4(hp)2Apm Bpm 2B4 2B4(78) Bt4 GBT,2567,NaN,16,16


In [113]:
median_df=df[df["capacity"].notnull()]
median=np.median(df1['capacity'])

In [ ]:
def fill_capacity(row):
    if row["capacity"].isnull():
        train=row["train_nr"]
        formation = df_jahresformation[df_jahresformation["Zug"]==train]
        capacity=formation["capacity"]
    else: capacity = median
return capacity

In [127]:
row = df.iloc[9]
train=row["train_nr"]
formation = df_jahresformation[df_jahresformation["Zug"]==train]
print(formation.first)

capacity=formation["capacity"]


528
<bound method NDFrame.first of         Zug  ZNS    VP  Debicode Zugart Suf. Traktion Bhf von  Ländercode UIC  \
8866    528    4    15      1102     IC  NaN      NaN     ZUE              85   
8868    528    4    15      1102     IC  NaN      NaN     ZUE              85   
8870    528    4    67      1102     IC  NaN      NaN     ZUE              85   
12836   528    4 -2294      1102     IC  NaN      NaN     ZUE              85   
12838   528    4 -2294      1102     IC  NaN      NaN     ZUE              85   
20281   528    4 -2294      1102     IC  NaN      NaN     ZUE              85   
20283   528    4 -2282      1102     IC  NaN      NaN     ZUE              85   
20285   528    4 -2282      1102     IC  NaN      NaN     ZUE              85   
20287   528    4 -2282      1102     IC  NaN      NaN     ZUE              85   
24529   528    4    15      1102     IC  NaN      NaN     ZUE              85   
24531   528    4    67      1102     IC  NaN      NaN     ZUE             

KeyError: 'capacity'